<a href="https://colab.research.google.com/github/IFuentesSR/RSE_drought/blob/main/AVHRR_MODIS_NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook pour le calcul des indices et anomalies d'humidité du sol MAI GEE (SMAP)

### Définition des variables d'environnement

In [ ]:
import os

WRK_DIR = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil/EO4DM')
os.chdir(WRK_DIR)
WRK_DIR = os.path.normpath('Y:/EO4DM')

TERRITORY = 'New Caledonia (Fr)'
DRIVE_FOLDER = 'EO4DM_EXPORT_NOTEBOOK'
CLEAN_RUNFOLDER = 0

TERRITORY_str = TERRITORY.replace(' ', '_').replace('(', '').replace(')', '')
DATA_HISTO = os.path.join(WRK_DIR,'DATA_HISTO',TERRITORY_str)

### Import des librairies

In [ ]:
import ee
import glob
import shutil
import folium
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import dmpipeline.GEE_Processing.GEE_generic_functions as geegen
import dmpipeline.GEE_Processing.gee_accounts as geeauth

### Authentification GEE

In [ ]:
path2key = os.path.dirname(geeauth.__file__)
project_id = geegen.googleAuthentification(path2key)

### Définition des fonctions

In [ ]:
def temporal_aggregation_SMAP(collection, image_ref, band):
    def inner(year):
        months = ee.List.sequence(1,12)
        def month_iteration(m):
            date = ee.Date.fromYMD(year, m, 1)
            mean = collection.select(band).filterDate(date, date.advance(1, 'month')).mean()
            mean = mean.reproject(image_ref.projection())
            return mean.setMulti({'system:time_start': date.millis(), 'system:index':date.format('YYYY_MM_dd'), 'month':ee.Number(m)})
        return months.map(month_iteration)
    return inner


def join(collection):
    def wrap(img):
        date = img.get('system:time_start')
        img_retrieve = img.addBands(collection.filter(ee.Filter.eq('system:time_start', date)).first().rename('MOD'))
        return img_retrieve.copyProperties(img, ['system:time_start', 'system:index', 'month'])
    return wrap


def sample_collection(geo):
    def wrap(img):
        date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
        idx = img.reduceRegion('first', geo, 9000).values().get(0)
        return ee.Feature(None, {'date':date, 'idx':idx})
    return wrap


### Appel des collections

In [ ]:
sm_param = 'soil_moisture_am'
smap_collection = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005")
date_start = smap_collection.limit(1, 'system:time_start', True).first().date()
date_end = smap_collection.limit(1, 'system:time_start', False).first().date()

geo = ee.Geometry.Point([165.4038, -21.5779])
months = ee.List.sequence(1,12)
years = ee.List.sequence(date_start.get('year'), date_end.get('year'))

## Aggrégation mensuelle des données

In [ ]:
smap_agg = years.map(temporal_aggregation_SMAP(smap_collection, smap_collection.first(), sm_param))
smap_agg = ee.ImageCollection.fromImages(smap_agg.flatten())

# print(smap_agg.limit(1, 'system:time_start', True).first().date().format('YYYY-MM-dd').getInfo())
# print(smap_agg.limit(1, 'system:time_start', False).first().date().format('YYYY-MM-dd').getInfo())

smap_agg = smap_agg.filterDate(date_start, date_end)

date_start_smap_agg = smap_agg.limit(1, 'system:time_start', True).first().date().format('YYYY-MM-dd').getInfo()
date_end_smap_agg = smap_agg.limit(1, 'system:time_start', False).first().date().format('YYYY-MM-dd').getInfo()
print(f'date_start_smap_agg = {date_start_smap_agg}')
print(f'date_end_smap_agg = {date_end_smap_agg}')


### Affichage pour une date spécifique

In [ ]:
month = 4
year = 2015

smap_date = smap_agg.filterDate(ee.Date.fromYMD(year, month, 1),
                                ee.Date.fromYMD(year, month+1, 1)).first()
date_1 = smap_date.date().format('YYYY-MM-dd').getInfo()

SMAP_ex = smap_date.getMapId({'min': 0, 'max': 1, 'palette': 'FF0000, FFFFFF, 0000FF', 'opacity':0.4})
centroid = geo.coordinates().getInfo()[::-1]
map = folium.Map(centroid, zoom_start=5)
folium.TileLayer(
    tiles=SMAP_ex['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'SMAP {date_1}',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Extraction sur un lieu spécifique

In [ ]:
samples = smap_agg.map(sample_collection(geo))
samples = samples.getInfo()
dates = [pd.to_datetime(n['properties']['date']) for n in samples['features']]
sm = [[n['properties']['idx']] for n in samples['features']]
plt.plot(dates, sm)

### Définition des fonctions pour le calcul des anomalies

In [ ]:
def set_month(img):
    return img.set('month', ee.Date(img.get('system:time_start')).get('month'))


def aggregation(collection, scale):
    def inner(date):
        date0 = ee.Date(date)
        img = ee.Image(collection.filter(ee.Filter.eq('system:time_start', date)).first())
        props = img.propertyNames()
        condition = ee.Algorithms.If(ee.Number(ee.ImageCollection(collection.filterDate(date0.advance(ee.Number(scale-1).multiply(-1), 'month'), date0.advance(1, 'month'))).size()).lt(scale),
                                     ee.Image().set('drop',1),
                                     ee.Image(collection.filterDate(date0.advance(ee.Number(scale-1).multiply(-1), 'month'), date0.advance(1, 'month')).mean()).set('drop',0))
        return ee.Image(condition).copyProperties(img, props).set('month', date0.get('month'))
    return inner


def z_scores(collection):
    def wrap(i):
        img = ee.Image(i)
        props = img.propertyNames()
        coll = collection.filter(ee.Filter.eq('month', i.get('month')))
        mean = ee.ImageCollection(coll).mean()
        sdv = ee.ImageCollection(coll).reduce(ee.Reducer.stdDev())
        return img.subtract(mean).divide(sdv).copyProperties(img, props)
    return wrap

### Creation des séries d'anomalies (MAI)

In [ ]:
smap_agg = smap_agg.map(set_month)
dates = smap_agg.aggregate_array('system:time_start')
smap_collection_2 = dates.map(aggregation(smap_agg, 1))
smap_collection_2 = smap_collection_2.filter(ee.Filter.eq('drop', 0))
smap_collection_2 = ee.ImageCollection.fromImages(smap_collection_2)
MAI_collection = smap_collection_2.map(z_scores(smap_collection_2))

date_start_mai = MAI_collection.limit(1, 'system:time_start', True).first().date().format('YYYY-MM-dd').getInfo()
date_end_mai = MAI_collection.limit(1, 'system:time_start', False).first().date().format('YYYY-MM-dd').getInfo()
print(f'date_start_mai = {date_start_mai}')
print(f'date_end_mai = {date_end_mai}')

In [ ]:
# N = dates.size().getInfo()
# for i in range(N):
#     print(ee.Date(dates.get(i)).format('YYYY-MM-dd').getInfo()) 

### Affichage des anomalies humidités pour une date spécifique

In [ ]:
month = 5
year = 2015

MAI_date = MAI_collection.filterDate(ee.Date.fromYMD(year, month, 1),
                                     ee.Date.fromYMD(year, month+1, 1)).first()

date_1 = MAI_date.date().format('YYYY-MM-dd').getInfo()

smap_date = smap_agg.filterDate(ee.Date.fromYMD(year, month, 1),
                                ee.Date.fromYMD(year, month+1, 1)).first()

In [ ]:
MAI_ex = MAI_date.getMapId({'min': -1, 'max': 1, 'palette': 'FF0000, FFFFFF, 0000FF', 'opacity':0.4})
SMAP_ex = smap_date.getMapId({'min': 0, 'max': 1, 'palette': 'FF0000, FFFFFF, 0000FF', 'opacity':0.4})
centroid = geo.coordinates().getInfo()[::-1]
map = folium.Map(location=centroid, zoom_start=7)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=SMAP_ex['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'SMAP {date_1}',
  ).add_to(map)
folium.TileLayer(
    tiles=MAI_ex['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'MAI {date_1}',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Extraction des anomalies sur un lieu spécifique

In [ ]:
samples = MAI_collection.map(sample_collection(geo))
samples = samples.getInfo()
dates = [pd.to_datetime(n['properties']['date']) for n in samples['features']]
ano = [n['properties']['idx'] for n in samples['features']]
df = pd.DataFrame(data={'idx':ano, 'date':dates})
d = np.zeros(len(ano))

In [ ]:
fig, axs = plt.subplots(figsize=(10, 3))
axs.fill_between(dates, d, ano, where=df['idx']>=d, interpolate=True, color='blue', alpha=0.3, label='MAI')
axs.fill_between(dates, d, ano, where=df['idx']<=d, interpolate=True, color='red', alpha=0.3)
axs.set_ylabel('MAI')
fig.show()

### Prépare export de la collection MAI

In [ ]:
# --- Prepare main folder name ---
date_start_str = date_start_mai.replace('-', '')
date_end_str = date_end_mai.replace('-', '')

# --- Generate directory ---
if (CLEAN_RUNFOLDER is None) or (CLEAN_RUNFOLDER==''): CLEAN_RUNFOLDER = 0
else: CLEAN_RUNFOLDER = int(CLEAN_RUNFOLDER)

outdir = os.path.join(WRK_DIR, f'RUN_MAI_DROUGHT_{TERRITORY_str}_{date_start_str}_{date_end_str}')
os.umask(0) # used to reset the directories permission
if not os.path.exists(outdir):
    os.makedirs(outdir)
    os.chmod(outdir, 0o777)
elif CLEAN_RUNFOLDER==1:
    shutil.rmtree(outdir)
    os.makedirs(outdir)
    os.chmod(outdir, 0o777)

# --- Generate sub-directories ---
outdir_mai = os.path.normpath(outdir + os.sep + 'MAI')
outdir_maimonth = os.path.normpath(outdir_mai + os.sep + 'MONTH/')
outdir_maistats = os.path.normpath(outdir_mai + os.sep + 'STATS/')
if not os.path.exists(outdir_mai): os.makedirs(outdir_mai)
if not os.path.exists(outdir_maimonth): os.makedirs(outdir_maimonth)
if not os.path.exists(outdir_maistats): os.makedirs(outdir_maistats)

# --- Concenate dir paths ---
OUTDIR_PATHS = (outdir_mai, outdir_maimonth, outdir_maistats)


In [ ]:
# --- Extract VHI bounding box ---
histo_files = glob.glob(os.path.join(DATA_HISTO,'0_INDICES', 'MODIS', 'DECADE','*.tif'))
with rasterio.open(histo_files[0]) as d_ds :
    (lon_min_modis, lat_min_modis,
     lon_max_modis, lat_max_modis) = d_ds.bounds

CRS_OUT = 'EPSG:4326'
SCALE_OUT = smap_collection.first().select(sm_param).projection().nominalScale()
GRID_OUT = ee.Geometry.BBox(float(lon_min_modis), float(lat_min_modis),
                            float(lon_max_modis), float(lat_max_modis))

# MAI_collection = MAI_collection.filterDate(CONFIG['PERIOD_START'], CONFIG['PERIOD_END'])
N_mai = MAI_collection.size().getInfo()
MAI_list = MAI_collection.toList(N_mai)

In [ ]:
for i in tqdm(range(N_mai), desc='MAI EXPORTED'):
  MAI_i = ee.Image(MAI_list.get(i))
  date_i = MAI_i.date().format('YYYYMM').getInfo()
  MAI_filename = f'MAI_{date_i}M'
  geegen.exportImage(DRIVE_FOLDER,
                     MAI_i,
                     MAI_filename,
                     export_folder=OUTDIR_PATHS[1],
                     path2key=path2key,
                     data_crs=CRS_OUT,
                     data_scale=SCALE_OUT,
                     data_region=GRID_OUT)
  del MAI_i,date_i,MAI_filename